# NGCF: a baseline implementation

## 1. Imports

In [1]:
import torch
import torch.nn as nn
from torch.nn import Module
from scipy.sparse import coo_matrix
from scipy.sparse import vstack
from scipy import sparse
import numpy as np
import pandas as pd
from numpy import diag
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.nn import MSELoss
from os import path
import io

In [2]:
from google.colab import files
uploaded = files.upload()

Saving data_train.csv to data_train.csv


## 2. Utils

### 2.1. Data loader

In [3]:
# load 100k data

def extract_users_items_predictions(data_pd):
    users, movies = \
        [np.squeeze(arr) for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    predictions = data_pd.Prediction.values
    return users, movies, predictions

def load100KRatings():
    data_pd = pd.read_csv(io.BytesIO(uploaded['data_train.csv']))
    print("BEFORE: ", data_pd.shape)
    users, movies, ratings = extract_users_items_predictions(data_pd)
    df = pd.DataFrame({'userId': users, 'itemId': movies, 'rating': ratings})
    #df = pd.read_table(path100k+r'\u.data',sep='\t',names=['userId','itemId','rating','timestamp'])
    return df


# movielens 1k
class ML1K(Dataset):

    def __init__(self,rt):
        super(Dataset,self).__init__()
        self.uId = list(rt['userId'])
        self.iId = list(rt['itemId'])
        self.rt = list(rt['rating'])

    def __len__(self):
        return len(self.uId)

    def __getitem__(self, item):
        return (self.uId[item],self.iId[item],self.rt[item])



### 2.2. Early stopping

## 3. Model

In [4]:
class GNNLayer(Module):

    def __init__(self,inF,outF):

        super(GNNLayer,self).__init__()
        self.inF = inF
        self.outF = outF
        self.linear = torch.nn.Linear(in_features=inF,out_features=outF)
        self.interActTransform = torch.nn.Linear(in_features=inF,out_features=outF)

    def forward(self, laplacianMat,selfLoop,features):
        # for GCF ajdMat is a (N+M) by (N+M) mat
        # laplacianMat L = D^-1(A)D^-1
        L1 = laplacianMat + selfLoop
        L2 = laplacianMat#.cuda()
        L1 = L1#.cuda()
        inter_feature = torch.sparse.mm(L2,features)
        inter_feature = torch.mul(inter_feature,features)

        inter_part1 = self.linear(torch.sparse.mm(L1,features))
        inter_part2 = self.interActTransform(torch.sparse.mm(L2,inter_feature))

        return inter_part1+inter_part2

class GCF(Module):

    def __init__(self,userNum,itemNum,rt,embedSize=100,layers=[100,80,50],useCuda=False):

        super(GCF,self).__init__()
        self.useCuda = useCuda
        self.userNum = userNum
        self.itemNum = itemNum
        self.uEmbd = nn.Embedding(userNum,embedSize)
        self.iEmbd = nn.Embedding(itemNum,embedSize)
        self.GNNlayers = torch.nn.ModuleList()
        self.LaplacianMat = self.buildLaplacianMat(rt) # sparse format
        self.leakyRelu = nn.LeakyReLU()
        self.selfLoop = self.getSparseEye(self.userNum+self.itemNum)

        self.transForm1 = nn.Linear(in_features=layers[-1]*(len(layers))*2,out_features=64)
        self.transForm2 = nn.Linear(in_features=64,out_features=32)
        self.transForm3 = nn.Linear(in_features=32,out_features=1)

        for From,To in zip(layers[:-1],layers[1:]):
            self.GNNlayers.append(GNNLayer(From,To))

    def getSparseEye(self,num):
        i = torch.LongTensor([[k for k in range(0,num)],[j for j in range(0,num)]])
        val = torch.FloatTensor([1]*num)
        return torch.sparse.FloatTensor(i,val)

    def buildLaplacianMat(self,rt):

        rt_item = rt['itemId'] + self.userNum
        uiMat = coo_matrix((rt['rating'], (rt['userId'], rt['itemId'])))

        uiMat_upperPart = coo_matrix((rt['rating'], (rt['userId'], rt_item)))
        uiMat = uiMat.transpose()
        uiMat.resize((self.itemNum, self.userNum + self.itemNum))

        #print(uiMat_upperPart)
        #print(uiMat.shape)

        A = sparse.vstack([uiMat_upperPart,uiMat])
        selfLoop = sparse.eye(self.userNum+self.itemNum)
        sumArr = (A>0).sum(axis=1)
        diag = list(np.array(sumArr.flatten())[0])
        diag = np.power(diag,-0.5)
        D = sparse.diags(diag)
        L = D * A * D
        L = sparse.coo_matrix(L)
        row = L.row
        col = L.col
        i = torch.LongTensor([row,col])
        data = torch.FloatTensor(L.data)
        SparseL = torch.sparse.FloatTensor(i,data)
        return SparseL

    def getFeatureMat(self):
        uidx = torch.LongTensor([i for i in range(self.userNum)])
        iidx = torch.LongTensor([i for i in range(self.itemNum)])
        if self.useCuda == True:
            uidx = uidx.cuda()
            iidx = iidx.cuda()

        userEmbd = self.uEmbd(uidx)
        itemEmbd = self.iEmbd(iidx)
        features = torch.cat([userEmbd,itemEmbd],dim=0)
        return features

    def forward(self,userIdx,itemIdx):

        itemIdx = itemIdx + self.userNum
        userIdx = list(userIdx)#.cpu().data)
        itemIdx = list(itemIdx)#.cpu().data)
        # gcf data propagation
        features = self.getFeatureMat()
        finalEmbd = features.clone()
        for gnn in self.GNNlayers:
            features = gnn(self.LaplacianMat,self.selfLoop,features)
            features = nn.ReLU()(features)
            finalEmbd = torch.cat([finalEmbd,features.clone()],dim=1)

        userEmbd = finalEmbd[userIdx]
        itemEmbd = finalEmbd[itemIdx]
        embd = torch.cat([userEmbd,itemEmbd],dim=1)

        embd = nn.ReLU()(self.transForm1(embd))
        embd = self.transForm2(embd)
        embd = self.transForm3(embd)
        prediction = embd.flatten()

        return prediction

## 4. Main

In [5]:
rt = load100KRatings()
userNum = rt['userId'].max() + 1
itemNum = rt['itemId'].max() + 1

rt['userId'] = rt['userId']
rt['itemId'] = rt['itemId']

rt

BEFORE:  (1176952, 2)


,userId,itemId,rating
0,43,0,4
1,60,0,3
2,66,0,4
3,71,0,3
4,85,0,5
...,...,...,...
1176947,9989,999,4
1176948,9991,999,5
1176949,9993,999,3
1176950,9996,999,4


In [9]:
para = {
    'epoch':60,
    'lr':0.01,
    'batch_size':2048,
    'train':0.8,
    'val': 0.2,
    'patience': 3 # number of epochs we are willing to wait 
}

ds = ML1K(rt)
# Get test set
trainLen = int(para['train']*len(ds))
train, test = random_split(ds, [trainLen,len(ds)-trainLen])

# Get validation and training set
valLen = int(para['val']*len(train))
validation, train = random_split(train, [valLen,len(train)-valLen])

dl = DataLoader(train,batch_size=para['batch_size'],shuffle=True,pin_memory=True)

model = GCF(userNum, itemNum, rt, 64, layers=[64,64,64])#.cuda()
# model = SVD(userNum,itemNum,50).cuda()
# model = NCF(userNum,itemNum,64,layers=[128,64,32,16,8]).cuda()
optim = Adam(model.parameters(), lr=para['lr'],weight_decay=0.001)
lossfn = MSELoss()

min_val_loss = 1e3
wait_param = 0
for i in range(para['epoch']):
    # Training loop
    for id,batch in enumerate(dl):
        #print('epoch:',i,' batch:',id)
        optim.zero_grad()
        prediction = model(batch[0], batch[1])#.cuda()
        loss = lossfn(batch[2].float(),prediction)#.cuda()
        loss.backward()
        optim.step()
    
    # Check whether early stopping is needed after epoch n.6
    valdl = DataLoader(validation,batch_size=len(validation),)
    for data in valdl:
        prediction = model(data[0],data[1])#.cuda()
    loss_val = lossfn(data[2].float(),prediction)

    print('epoch:', i ,'train-loss:', loss, 'val-loss:', loss_val)

    if i > 5:
      if loss_val < min_val_loss:
        min_val_loss = loss_val
        wait_param = 0
      else:
        wait_param += 1
      if wait_param >= para['patience']:
        break

testdl = DataLoader(test,batch_size=len(test),)
for data in testdl:
    prediction = model(data[0],data[1])#.cuda()

loss = lossfn(data[2].float(),prediction)

print(prediction)
print(data[2].int())
accuracy = 0
for i in range(len(prediction)):
  if prediction[i] >= data[2].float()[i] - 0.5 and prediction[i] <= data[2].float()[i] + 0.5:
    accuracy += 1
accuracy = accuracy/len(test)

print('test-loss:', loss, 'accuracy:', accuracy)

epoch: 0 train-loss: tensor(0.9665, grad_fn=<MseLossBackward>) val-loss: tensor(1.0470, grad_fn=<MseLossBackward>)
epoch: 1 train-loss: tensor(1.0503, grad_fn=<MseLossBackward>) val-loss: tensor(1.0355, grad_fn=<MseLossBackward>)
epoch: 2 train-loss: tensor(1.0801, grad_fn=<MseLossBackward>) val-loss: tensor(1.0284, grad_fn=<MseLossBackward>)
epoch: 3 train-loss: tensor(1.0556, grad_fn=<MseLossBackward>) val-loss: tensor(1.0446, grad_fn=<MseLossBackward>)
epoch: 4 train-loss: tensor(1.0186, grad_fn=<MseLossBackward>) val-loss: tensor(1.0202, grad_fn=<MseLossBackward>)
epoch: 5 train-loss: tensor(1.0279, grad_fn=<MseLossBackward>) val-loss: tensor(1.0194, grad_fn=<MseLossBackward>)
epoch: 6 train-loss: tensor(1.0252, grad_fn=<MseLossBackward>) val-loss: tensor(1.0192, grad_fn=<MseLossBackward>)
epoch: 7 train-loss: tensor(0.9781, grad_fn=<MseLossBackward>) val-loss: tensor(1.0656, grad_fn=<MseLossBackward>)
epoch: 8 train-loss: tensor(1.0119, grad_fn=<MseLossBackward>) val-loss: tensor(